In [1]:
import json
import jsonschema
import random
import itertools
import numpy as np
from jsonschema import validate
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, ReadoutError, depolarizing_error
from qiskit.circuit.random import random_circuit
from mitiq import zne, ddd
from mitiq.zne.scaling.folding import fold_global, fold_gates_at_random, fold_all
from mitiq.zne.scaling.layer_scaling import get_layer_folding
from mitiq.zne.scaling.identity_insertion import insert_id_layers
from mitiq.benchmarks.ghz_circuits import generate_ghz_circuit
from qiskit.circuit import Gate
from collections import defaultdict
import functools
import tomlkit
import toml

# ZNE:

In [2]:
# Load Schema:

def load(schema_path):
    with open(schema_path, 'r') as file:
        return json.load(file)
    
zne_schema =load('./schema/zne_schema.json')

In [3]:
# Create a test "batch" experiment:

zne_batch_test = {"noise_scaling_factors": [[1, 1.25, 1.5], [1,2,3], [2,4,6]],        # Noise scaling values
    "noise_scaling_method": ["global"],                                               # Folding method
    "extrapolation": ["polynomial", "linear"],                                        # Extrapolation method
    }

## Deconstructing a Batch Dictionary:

In [4]:
# Create single experiments from batch object:

# Define a function to make all combinations of experiments from a "batch dictionary" which should be formatted like the test case above
def make_experiment_list(batch_dict):

    # Initialize empty list where we will append each new experiment
    exp_list = []

    # Make list with all combinations of key values from our "batch dictionary"
    combo_list = list(itertools.product(batch_dict['noise_scaling_factors'], 
                                        batch_dict['noise_scaling_method'], 
                                        batch_dict['extrapolation']))
    # Iterate over the list
    for k in range(len(combo_list)):

        # Initialize single experiment dictionary with keys
        exp = {x: set() for x in ['noise_scaling_factors', 'noise_scaling_method', 'extrapolation']}

        # Map each key to its unique value from our list of combinations
        exp['noise_scaling_factors'] = combo_list[k][0]
        exp['noise_scaling_method'] = combo_list[k][1]
        exp['extrapolation'] = combo_list[k][2]

        # Pull out each experiment
        exp_list.append(exp)

    # Returns a list of experiments
    return exp_list

In [5]:
# Implementing our function using the test case defined above:

make_experiment_list(zne_batch_test)

[{'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'}]

## Validating a batch of experiments:

In [6]:
# Validate each experiment in our batch object:

# Create function that takes in a batch dictionary and schema to validate against
def batch_validate(batch_dict, schema):

    # Initialize empty list for validation results
    validation_results = []

    # Create list of experiments
    formatted_batch = make_experiment_list(batch_dict)

    # Iterate over list of experiments and individually validate
    for k in formatted_batch:
        try:
            validate(instance=k, schema=schema)
            result = "validation passed"
        except jsonschema.exceptions.ValidationError as e:
            result = "validation failed"

        # Pull out validation results
        validation_results.append(result)
        
    return validation_results

In [7]:
# Implementing our batch validate function using the same test case:

batch_validate(zne_batch_test, zne_schema)

['validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed']

In [8]:
# Ella's basic noise model:

def basic_noise(n_qubits, prob=0.005):

    noise_model = NoiseModel()

    for i in range(n_qubits):
        readout_err = ReadoutError([[0.99, 0.01], [0.2, 0.8]])
        noise_model.add_readout_error(readout_err, [i])
        
    depolarizing_err1 = depolarizing_error(prob, num_qubits=1)
    depolarizing_err2 = depolarizing_error(prob, num_qubits=2)
    noise_model.add_all_qubit_quantum_error(depolarizing_err1, ['h', 'x', 'y', 'z'])
    noise_model.add_all_qubit_quantum_error(depolarizing_err2, ['cx'])

    return noise_model

## Overhead Related Functions:

In [9]:
# Single experiment executor (very slightly modified from Ella's):
# Likely to be unused, but just in case:

def execute_no_overhead(circuit, backend, shots):       # Set noise_model to thermal, basic, noiseless
    circ = circuit.copy()
    circ.measure_all()
    num_qubits = circ.num_qubits
    
    transpiled_circuit = transpile(circ, optimization_level=0, backend=backend)

    results = backend.run(transpiled_circuit, optimization_level=0, shots=shots).result()
    counts = results.get_counts(transpiled_circuit)

    total_shots = sum(counts.values())
    probabilities = {state: count / total_shots for state, count in counts.items()}

    expectation_value = probabilities['0'*num_qubits]+probabilities['1'*num_qubits]
    return expectation_value

In [10]:
def get_original_circ_counts(circuit, backend, shots):

    circ = circuit.copy()
    circ.measure_all()
    num_qubits = circ.num_qubits

    og_params = {x: set() for x in ['depth', '1qbit_gates', '2qbit_gates', 'exp_val']}

    transpiled_circuit = transpile(circ, optimization_level=0, backend=backend)

    og_params["depth"] = transpiled_circuit.depth()
    counts = defaultdict(int)

    for inst in transpiled_circuit.data:
        if isinstance(inst.operation, Gate):
            counts[len(inst.qubits)] += 1
                
        o = {k: v for k, v in counts.items()}
                        
        single_qubit_gates = (o.get(1)) if o.get(1) is not None else 0
        two_qubit_gates = (o.get(2)) if o.get(2) is not None else 0
                
        og_params["1qbit_gates"] = single_qubit_gates
        og_params["2qbit_gates"] = two_qubit_gates 

    results = backend.run(transpiled_circuit, optimization_level=0, shots=shots).result()
    counts = results.get_counts(transpiled_circuit)

    total_shots = sum(counts.values())
    probabilities = {state: count / total_shots for state, count in counts.items()}

    expectation_value = probabilities['0'*num_qubits]+probabilities['1'*num_qubits]
    og_params["exp_val"] = expectation_value
    
    return og_params

In [11]:
def scaled_overhead(circuit, backend, shots):

    og_params = get_original_circ_counts(circuit, backend, shots)

    inst = schema_executors.overhead_executors()
    
    mit_params = inst.write_metadata(circuit=circuit, shots=shots, backend=backend)

    scaled_params = {x: set() for x in ['add_circs', 'add_depth', 'add_1qbit_gates', 'add_2qbit_gates', 'exp_val_improvement']}

    scaled_params["add_circs"] = mit_params["mit_circs"]
    scaled_params["add_depth"] = mit_params["mit_depth"] - og_params["depth"]
    scaled_params["add_1qbit_gates"] = mit_params["mit_1qbit_gates"] - og_params["1qbit_gates"]
    scaled_params["add_2qbit_gates"] = mit_params["mit_2qbit_gates"] - og_params["2qbit_gates"]
    scaled_params["add_1qbit_gates"] = mit_params["mit_1qbit_gates"] - og_params["1qbit_gates"]
    scaled_params["exp_val_improvement"] = og_params["exp_val"]/mit_params["mit_exp_val"]

    return scaled_params

# Testing:

In [12]:
# Making a 3-qubit test GHZ circuit:

n = 3
ghz_circ = generate_ghz_circuit(n, return_type='qiskit')

ghz_circ.x([0,1])
ghz_circ.x([0,1])
ghz_circ.y([1,2])
ghz_circ.y([1,2])

print(ghz_circ)

     ┌───┐     ┌───┐┌───┐               
q_0: ┤ H ├──■──┤ X ├┤ X ├───────────────
     └───┘┌─┴─┐└───┘├───┤┌───┐┌───┐┌───┐
q_1: ─────┤ X ├──■──┤ X ├┤ X ├┤ Y ├┤ Y ├
          └───┘┌─┴─┐├───┤├───┤└───┘└───┘
q_2: ──────────┤ X ├┤ Y ├┤ Y ├──────────
               └───┘└───┘└───┘          


In [14]:
# import schema_executors
from schema_executors import OverheadExecutors

inst = OverheadExecutors()

test_backend = AerSimulator(noise_model=basic_noise(ghz_circ.num_qubits))

get_original_circ_counts(ghz_circ, test_backend, 1000)

test_path = '.test.toml'

metadata = inst.write_metadata(circuit=ghz_circ, shots=1000, backend=test_backend)
print(metadata)

{'mit_circs': 3, 'mit_1qbit_gates': 53, 'mit_2qbit_gates': 12, 'mit_depth': 44, 'mit_exp_val': 0.5899999999999986}
